# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='./Data/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight;")

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    current_height = browser.execute_script("return document.body.scrollHeight;")
    
    if current_height == last_height:
        print('到達頁面底端')
        break
    else:
        print('網頁更新中...')
        last_height = current_height
        continue

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 年終獎金來了…43.2%上班族選擇先存一半！壽險專家曝光「高CP值規劃3重點」技巧

▲壽險業者建議，拿到年終獎金除了發給長輩、晚輩紅包外，也要好好規劃自己的未來。（圖／全球人壽提供）
記者楊絡懸／台北報導
今年春節假期較早，各公司員工也非常期待年終獎金，調查指出，43.2%上班族會選擇先存一半年終獎金，執行財務分配。壽險業者建議，拿到年終獎金除了發給長輩、晚輩紅包外，也要好好規劃自己的未來，透過「高CP值規劃3重點」，為自己累積退休財源或實現未來的財務目標。
根據人力網站「企業年終獎金與勞工尾牙期望調查」指出，若年終獎金不理想，有52.2%的受訪上班族「會萌生辭意」；若有拿到年終獎金，有43.2%的受訪上班族會將年終獎金存一半起來。對初入職場的社會新鮮人而言，年終獎金可說是難得的「小確幸」。
全球人壽建議，20至35歲之間的青年族群，最好及早進行定期定額累積資產，建構防護網；35至45歲，則重在工具、選對方法放大資產、完善保障；這兩個階段可運用「利變型增額壽險」或「投資型保險」，同時達到有紀律累積資產及放大資產效果。
45歲以上的民眾，則重在防守，穩健規劃、打造現金流。全球人壽認為，這階段可選擇兼具有穩定成長及確定現金流特性的商品，比如未來提供轉入即期年金的「利變型增額壽險」或「還本型壽險」。
全球人壽指出，首先，相對穩健的「利變型增額壽險」是適合做為退休財源的工具，因為風險較低、穩定性高，且繳費年期多元，可長、可短，可依個人需求進行配置；若有長期財務目標，就可選擇繳費年期較長的利變型增額壽險。
其次，當宣告利率大於預定利率時，就有機會可領到增額回饋分享金，投保前6年只能加購保額或累積生息，第7年起即可領到現金；如果有外幣配置需求，也可以透過美元計價的利變型保單進行規劃。
在規劃這類保障時，要注意是否有豁免保險的功能。全球人壽表示，原因是，許多人在準備退休金時，只著重如何有效率的累積、放大資產，卻忽略萬一發生意外導致被迫提早退休的風險，可能連原本規劃好的退休計劃都必須中斷，嚴重影響家庭及生活；如果有豁免保險機制，萬一發生特定疾病或失能，不但不用再繳保費，保險公司還會協助繼續繳完剩下的保費，讓退休計畫不中斷。
▲全球

▲ 民眾可以趁匯價低點換日圓。（示意圖／資料照）
記者李瑞瑾／台北報導
日圓匯價又刷新1年多來新低紀錄！根據台銀今（13）日上午牌告的日圓現金賣出匯價，日圓來到0.2765元，再次改寫一年以來新低紀錄。
日幣兌美元匯率持續貶值，根據彭博即時匯率，截至今天上午9點32分，日幣兌美元來到109.62日圓兌1美元，是去年12月26日匯率109.63以來新低。
日圓本身不斷貶值，加上新台幣維持強勢，讓日圓匯價變得更便宜了！根據台銀今日上午牌告的現金賣出匯價，日圓來到0.2765元，是前（2018）年12月17日（匯價0.2757元）、一年一個月以來的新低價位。
若以兌換新台幣5萬元來看，今天可換到18萬832日圓，相較去年8月13日的匯價0.3029，當時只能換到16萬5,071日圓，就相差了1萬5,761日圓，等於多賺到了兩張迪士尼門票！
----------------------------------------------------------------------

[11] 新台幣早盤再度衝破30元關卡　一度升值逾1角

▲ 新台幣兌美元今天開盤升值0.9分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（13）天開盤升值0.9分、以30.01兌1美元開出，開盤之後再度突破30元關卡、來到29字頭，並一度升值超過1角。
新台幣兌美元上週五（10）日收盤在30.019兌1美元；今天開盤以30.01元開出、升值0.9分。
新台幣開盤後持續走升，截至上午9點12分，新台幣來到29.915元兌1美元，升值1.04角。
----------------------------------------------------------------------

[12] 台股上演謝票行情！開盤上漲44點　挑戰收復12,100點

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
總統大選在上週六（11日）正式落幕，由現任總統蔡英文取得連任，市場不確定性逐漸消散，分析師看好台股在選後能有亮眼表現。台股今（13）日開盤上漲44點、以12,069點開出。
台股開盤後持續走高，截至上午9點01分，指數來到12,097點、上漲72點。
電子三雄中，台積電（2330）以342元開出、上漲0.74%；鴻海（2317）開在89.7元、上漲0.79%；股王大

▲加油站 。（圖／記者張一中攝）
記者吳靜君／台北報導
中東地緣政治關係緊繃，中油今（12）日宣布，自明（13）日起汽柴油每公升調漲0.1元。調漲後零售價格為92無鉛汽油每公升27.8元、95無鉛汽油每公升29.3元、98無鉛汽油每公升31.3元、超級柴油每公升25.7元。
由於本周發生中東和美國的緊張氣氛升溫，國際油價也跟著浮動，外界也預期油價將會上揚。
台灣中油表示，受到美國與伊朗之間軍事警戒程度升級、波斯灣地區油輪輸運安全疑慮再起，帶動國際油價上揚。本週調價指標7D3B週均價為每桶67.84美元，較前週67.21美元上漲0.63美元，新台幣兌美元匯率升值0.054元，國內油價依公式計算漲幅為0.61％，92無鉛汽油及柴油均未超過亞鄰最低價，故汽、柴油應各調漲0.1元，足額調整。
調漲後參考零售價格分別為92無鉛汽油每公升27.8元、95無鉛汽油每公升29.3元、98無鉛汽油每公升31.3元、超級柴油每公升25.7元。但是中油強調，調幅大小完全取決於油價公式的計算結果，絕無「漲多跌少」或「只漲不跌」情事。國內所需的油源幾乎全從國外進口，來之不易，因此籲請消費者體認全球油源蘊藏量有限，珍惜資源、節約用油。
----------------------------------------------------------------------

[25] iPhone 舊換新優惠「縮水」！蘋果調降回收價格　秒看懂差多少

▲Apple直營店。（圖／記者林敬旻攝、示意圖）
記者吳靜君／台北報導
iPhone舊機換新機的優惠縮水了！蘋果官方網站宣布最新10款iPhone6S到iPhone Xs Max舊機換新機的折抵價格，回收的折抵價格從2200元到1萬5100元不等，但是相較於去年的折抵價格，優惠減少約2100元到近4760元不等。
蘋果官方網站公布最新的舊機換新機的折抵價格共計10款，包含iPhone Xs Max、iPhone Xs、iPhone XR、iPhoneX等到iPhone 6s的價格。
以iPhone Xs Max的換購價格為1萬5100元，但是如果與去年相比還可以折抵1萬7200元來看，縮水了將近2100元。而價格差異最大的是iPhone X，去年可抵的價格為1萬2460元，但是今年則已經降到7700元，顯示想用手中iPhone X換購新機的民眾

▲2020上半年全球經濟前景尚未明朗化，外銀點出5大投資主題，可望有強韌表現。（圖／pixabay）
記者紀佳妘／台北報導
今（2020）年全球經濟前景充滿不確定性，打亂投資人布局方向。滙豐私人銀行對此提出5大投資主題，預料這些主題將在上半年全球情勢尚未明朗的情況下，依然具有保持強韌的表現。
首先是亞洲消費動力，滙豐私人銀行北亞首席市場策略師何偉華指出，亞洲的富裕中產階級消費者數量龐大，可帶來具吸引力的結構性成長機遇。
何偉華表示，亞洲私人財富顯著累積、大陸空巢家庭數目上升，以及成長迅速的東協年輕城市人口，都是亞洲消費穩健增長的主要動力。雖然面對全球不明朗因素，但可在亞洲個人服務、高檔消費品、電子商務、娛樂、旅遊、教育、醫療保健和金融服務領域物色優質增長機會。
第二是重塑新大陸，把握大陸經濟轉型、科技升級、金融產業開放和刺激經濟政策帶來的結構增長機會。何偉華表示，大陸的半導體淨進口量龐大，總值2,270億美元占2018年進口總額21%，這是大陸積極尋求科技自主和升級的原因之一。
何偉華認為，科技業是大陸最大的單一進口產業，而台灣的半導體公司將受惠大陸對晶片急速成長的需求，其中包括明年第五代流動通訊（5G）的面世、更趨輕薄的智能手機和無人駕駛技術，都帶動對高端晶片和電子零組件需求的強勁成長，同時也看好受惠於大陸結構性改革和市場開放的公司，並在大陸金融股和偏向內需市場的股票尋求個別機遇，包括粵港澳大灣區優惠政策的股票等。
第三是亞洲企業債券機遇和新興市場債券以息差為重點，強勁的當地基本面和結構增長潛力，加上央行寬鬆政策，都對亞洲及新興市場美元債券和本地貨幣債券有利，相比已開發市場的債券，前者提供更具吸引力的追尋息差機會。
從資金流的技術分析來看，全球負收益債市總值高達11.7兆美元，各地追求收益的投資者將轉投較高收益的資產，因此亞洲和新興市場企業債券很可能吸引追求高收益的資金流入。
第四為亞洲高股息增長策略，何偉華指出，回顧過去十年，亞洲高息股的總回報勝於MSCI亞洲（日本除外）指數，預期具優質增長潛力的亞洲企業將更有可能維持未來配息，甚至調高配息率，讓高息成長股受惠環球追求收益的趨勢，支持其總回報上升。
第五是工業革命4.0，滙豐私人銀行強調，5G技術是第4次工業革命的關鍵部份，將可顯著提高網絡能力，為消費者和企業開拓嶄新功能，包括物聯網、自動駕駛汽車、遠距手術、智慧

▲測試龍頭京元電今日尾牙，高層主管上台敬酒。（圖／業者提供）
記者周康玉／台北報導
半導體測試龍頭京元電（2449）今（10）日於苗栗銅鑼二廠舉辦尾牙，席開近500桌，供應商、客戶與同仁齊聚一堂。尾牙節目於晚間6點在董事長李金恭先生宣布開始下熱鬧登場，尾牙現場氣氛十分熱烈，京元電子員工歡笑聲及掌聲不斷。
尾牙現場由吳怡霈擔任主持人，邀請藝人包含靈魂歌姬艾怡良、巴冷公主梁文音、清新歌手嚴爵及創作才子蔡旻佑到場表演，公司還準備了多份大獎，中獎率高達五成以上，最大獎也高達10萬元。
▼京元電尾牙，創作才子蔡旻佑到現場開唱。（圖／業者提供）
京元電董事長李金恭及總經理劉安炫現場致詞表示，對員工一年來努力成果都十分感謝，去（2019）年12月合併營收24.0億元，帶動去年第四季合併營收達71.5億元，集團全年營收255.4億元，年成長率達22.7%，第四季與全年營收皆創歷史新高。李金恭表示，期待京元電子今年能夠持續穩健成長，提供員工更美好的未來。
李金恭指出，在經營團隊及員工努力之下，京元電近年來業績及獲利持續成長，連續多年穩定加薪及發放獎金，並榮獲去年幸福企業大賞，為科技業之半導體界「前20大幸福企業」，完善獎金分紅制度，提供子女、配偶團保免費之福利，每年舉辦各項靜、動態活動、運動比賽等，幫助同仁舒壓放鬆，更是封測業唯一獲頒1次勞動部國家人才發展獎及4次勞動力發展署TTQS人才發展品質管理認證金牌之企業。
展望2020年，李金恭表示，公司已經在主流市場趨勢的5G、AI、IoT、資料中心與車用等市場提前成功卡位，因此未來幾年的營運績效仍可望穩健走揚。
京元電董事長李金恭本身就是苗栗客家人，為了推廣客家文化，回饋客家鄉親，選擇回到故鄉苗栗建廠，希望能替苗栗、竹南及銅鑼一帶增加更多的就業機會。
目前京元電子全球總員工數約9000人，光是苗栗地區就有7000多名員工，占全體總員工四分之三，為苗栗縣內最大企業之一。李金恭強調「愛鄉、愛土、愛苗栗」，在中央及地方政府支持下，加大力道投資苗栗並帶動經濟繁榮。
▼京元電董事長李金恭。（圖／業者提供）
----------------------------------------------------------------------

[48] 搶人才！中租營收表現佳　今年徵數位、國際人才600人

▲中租迪和今年徵才600人。（圖／

記者周康玉／台北報導
選前之夜，電子五哥緯創（3231）今（10）日舉行2019年終尾牙晚會，席開780桌，光是參與員工人數逾7800人。緯創董事長林憲銘每年都會親自抽出現金獎給幸運同仁，據了解，今日現金獎不會低於去年的60萬元，總金額至少破千萬。林憲銘上台致詞除了宣布今天獎金比去年好之外，還在結語時提醒同仁，明天一定要去投票，非常應景。
緯創尾牙於台北市南港展覽館舉辦，晚間6點登場，開場前一分鐘，還有倒數影片，並且在台上播放今年成果，由於聲光音效極為強烈，並且逐一回顧公司各部門去年的工作成果，被點到的部門員工歡呼連連，也成為晚會的熱烈開場。
緯創今年尾牙主題是「2020迎風而起 逆風高飛」，開場影片除了透露緯創去年在印度和越南等海外擴廠進度外，也特別透露，去年在筆電業務通過了五大品牌廠審查，今年產能會有翻倍成長。
林憲銘上台致詞表示，去（2019）年一開始就意識到是非常不平靜的一年，從總體經濟來看，加上區域政治衝突 ，貿易戰等， 讓去年全球經濟並沒有非常的順暢，因此他在去年初對於全年營運做了保守的估計。
林憲銘表示，一年來下來，經過同仁的努力，雖然在總體經濟不是很好，全年營收並沒有成長，但緯創的體質改變，去年第三季獲利就比去年同期有相當幅度的成長。 林憲銘表示，雖然他不能做預測，但他相信2019年的獲利會比前（2018）年成長。
至於員工最關心的現金獎，林憲銘也掛保證「今晚的抽獎比去年要多的，大獎比去年多；就連剛剛才核發的年終獎金，也比去年要多！」讓台下員工鼓掌叫好。
林憲銘也難得在公開場合談及員工調薪，他表示，除了每年四月份正常調薪以外，最近又推動了員工提撥公司持股的辦法，就是員工加薪3%，員工加薪同時可儲蓄，公司把營運成長跟同仁共享是領導層的責任。
緯創在去年獲得MSCI的雙A企業，林憲銘表示，包括台積電等一共才五個公司獲得此評選，是因為緯創在環保公益公司治理等各方面多年的努力，去年終於被看到，這些都是員工努力的成果。
林憲銘說，尾牙就是和員工說感謝的日子，並且特別在致詞結束前，提醒台下同仁「明天一定要去投票！」在選前之夜，特別點題。
----------------------------------------------------------------------

[56] 主計總處公布11月經常性薪資42,130元　年增率為2.16%　

▲ 行

▲台塑四寶業績說會，資料照。（圖／記者余弦妙攝）
記者余弦妙／台北報導
台塑四寶今（10）日舉行12月業績說會，並公布去年2019年財報成績單，合計四大公司（台塑、南亞、台化、台塑化），受美中貿易戰與國際油價波動衝擊，合計稅後純益1,271億元，比起前年的2,109億元，大幅縮水四成，其中台塑獲利年減26.1%，衰退幅度為四寶中最少；每股純益為5.86元，拿下台塑四寶「獲利王」頭銜。
台塑四寶去年合併營收1.46兆元，較前年1.73兆元減少15.6%。台塑指出，因液鹼、AN、MMA、ECH、AE及NBA等產品售價下跌，利差縮小，影響獲利表現。
而南亞表示，主要是各產品市場供給大多增加或持平，而因競爭激烈、產品價格偏低，部份產線不敷成本而停車減產，導致營收減少。其中以化工產品(EG及BPA為主)減少最多，其次為聚酯產品。全年稅後純益230.69億元，年減56.2%，衰退幅度是四寶最大，每股純益2.91元。
台化表示，主要ARO-3異常，後段PX製程僅開一線，PX減少外售及萃餘油等回售塑化減少共影響34億元；SM自用增加，減少外售4.5億元。導致獲利減少，去年稅後利益為299.91億元，年減37.9%，每股稅後盈餘5.14元。
----------------------------------------------------------------------

[70] 美時去年12月營收年增227.29%　全年營收年增43.86%創歷史新高

▲美時董事長韋斯曼（Robert Wessman）。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
美時化學製藥（1795）今（10）日公布2019年12月自結合併營收為17.34億元，較上月增加138.2％，較去年同期增加227.29%，全年自結合併營收為92.48億元，較2018年同期的64.28億元增加43.86%，創歷史新高。
美時表示，去年12月當月外銷出貨佔營收比重約7成，因此成為營收跳升的主要動能，至於全年營收的話，其中外銷比重略超過3成，而美國與非美國的占比約為8:2，成為區域比重最多元的一年。
美時指出，2019年全球外銷業務的成績主要建立於公司自主研發、生產且已在歐洲上市的血癌藥(Lenalidomide)、非小細胞肺癌藥物(Gefitinib)、乳癌藥物(Vinorelbine)，以及銷往美國的戒癮含片

▲ 台股在萬點~年線區間仍有支撐。（圖／記者湯興漢攝）
記者李瑞瑾／綜合報導
台股在中美將簽訂第一階段貿易協議激勵下，持續高檔震盪，展望後市，台新2000高科技基金經理人沈建宏表示，據歷史經驗，1~2月為作夢行情及個股表現格局，預期加權指數在萬點~年線附近，仍有強勁支撐，第一季指數仍可樂觀期待，建議可利用短線的急跌分批逢低布局。題材部分則看好成長性強勁的5G、半導體族群，另外，傳產股跟隨選前政策作多，亦具個別次族群行情，可留意相關政策受惠股。
沈建宏指出，分析目前國際環境有五大關注焦點：一、中美貿易戰將達成第一階段協議，但華為禁運令短期可能不會鬆綁，中美貿易戰對股市衝擊性降低；二、美伊衝突升溫，提供股市技術面漲多拉回理由，但不致扭轉股市方向。
第三，ECB將重啟QE，FED也恢復擴表，有助資金行情底部承接；四、中國製造業PMI指數止跌，美國製造業ISM指數第一季預估也可望止跌回溫；五、美國眾議院通過對川普彈劾的程序，但參議院為共和黨掌控,成案機率低。整體而言，國際系統風險不高。
沈建宏指出，雖然目前國際情勢仍在可控範圍，但後市盤面仍需留意以下趨勢，包含CES展及MWC展與第一季科技新產品發表市場反應；華為禁制令是否鬆綁或趨嚴；經濟數據是否繼續惡化等。若能朝正向發展，則台股後市反彈行情可期。
沈建宏表示，就基本面而言，預期2020年台股企業獲利恢復年正成長，成長幅度甚至有機會上看二位數，提供台股良好下檔支撐，看好五大布局方向。一、受惠庫存下滑及晶圓代工龍頭廠提高資本支出的半導體族群；二、受惠5G進度積極且提前的材料、RF、伺服器、骨幹網路等類股；三、受惠車用電子及汽車電子化趨勢持續的零組件；四、傳產股看好部分塑化股、航運、食品股等；五、高股息殖利率兼具成長性之次族群。
----------------------------------------------------------------------

[84] 王文淵加碼2萬紅包！連同年終獎金4.94個月　台塑集團工會滿意

▲台塑企業工會理事長吳祈忠。（圖／記者余弦妙攝）
記者余弦妙／台北報導
台塑集團總裁王文淵今（10）日與台塑企業工會面會，討論今年的年終獎金紅包會是多少，一早王文淵與工會理事長吳祈忠就進行會面討論，經過約莫2個半小時的討論過後，最終拍板今年台塑集團年終為4.94個月本薪+2萬元紅包。根據

In [6]:
# 關閉瀏覽器
browser.quit();